# 618 Project - Cosmetics Product Analysis Part 1

**Title:**

Uncovering Market Trends and Key Drivers of Customer Satisfaction: A Data Analysis of Sephora Products and Reviews

**Team Members**: 
- Ruizhe Wang (Unique Name: wrzwrz)
- Zhiyi Ji (Unique Name: jizhiyi)

## Overview

This project focuses on analyzing the **Sephora Products and Skincare Reviews Dataset**, which contains detailed information on over 8,000 beauty products and 1 million customer reviews. The dataset provides key insights into the cosmetic industry through product attributes, prices, ingredients, and customer feedback. 

Our primary objective is to identify **factors that drive higher product ratings** and understand how ingredients, prices, and product types influence customer satisfaction. Additionally, we aim to explore why certain products underperform by analyzing user reviews. Beyond ratings, we will investigate **trends in consumer sentiment** over time and assess the impact of movements like "clean beauty" and seasonal variations in feedback. 

Using **data manipulation, sentiment analysis, and visualization**, we intend to uncover patterns in customer behavior, empowering brands to align their strategies with customer needs and industry trends.

## Motivation

The cosmetic and skincare industry is evolving rapidly, with consumers becoming more focused on **tailored solutions and sustainable products**. Customer reviews are now essential in shaping brand reputations and guiding purchase decisions. This project is motivated by a desire to understand **what drives customer satisfaction**, allowing businesses to improve product offerings based on consumer needs. 

Specifically, we aim to address the following **3 real-world questions**:
1. **What factors drive higher product ratings?** Why do certain products underperform?  
   - What we hope to learn: Identify which product attributes (e.g., ingredients, price) are linked to higher satisfaction and uncover common issues affecting low-rated products. We also want to generate methods to resolve the issues. 

2. **How does customer sentiment change over time?**  
   - What we hope to learn: Analyze shifts in customer feedback and assess the impact of trends (e.g. "clean beauty") to better understand evolving preferences.  

3. **Are certain ingredients linked to positive or negative customer experiences?**  
   - What we hope to learn: Detect patterns in ingredient preferences and identify sources of customer dissatisfaction.  

Through this analysis, we hope to generate **actionable insights** that help brands meet customer expectations, avoid problematic ingredients, and **enhance customer experiences**. 

## Data Sources

Our project utilizes two primary datasets, which complement each other by linking **product attributes** with **customer feedback**, providing the comprehensive information for our analysis.  

This data was collected in March 2023 and provides detailed information on a variety of beauty and skincare products available on Sephora's online store. It includes both product details and customer reviews, offering a comprehensive view of customer feedback and product features. The dataset is publicly available on Kaggle, a popular data platform, making it suitable for exploratory analysis and machine learning tasks.

**URL:**
- [Kaggle: Sephora Products and Skincare Reviews Dataset](https://www.kaggle.com/datasets/nadyinky/sephora-products-and-skincare-reviews/data)

**Description:**

1. **Product Information Dataset** (`product_info.csv`)  
   - Contains detailed data on **8,000+ products** from various brands, covering the following attributes:
     - **Product name** and **brand**  
     - **Price**  
     - **Ingredients** and **product type** (e.g., moisturizer, serum)  
     - Product features (e.g., organic, vegan, chemical-free)  
   - **Purpose**: This dataset allows us to analyze product pricing, ingredients, average ratings and categorize product types.  

2. **Customer Reviews Dataset**  
   - Split across multiple files: `review_0-250.csv`, `review_250-500.csv`, `review_500-750.csv`, `review_750-1250.csv`, `review_1250-end.csv`.  
   - Contains **1 million+ reviews** on over **2,000 skincare products**, with key data fields:
     - **Ratings**: Numerical scores reflecting customer satisfaction  
     - **Review text**: Customer opinions, experiences, and feedback  
     - **Review metadata**: User details (e.g., skin type) and peer ratings on the helpfulness of reviews  
     - **Date**: Submitting date of the reviews. 
   - **Purpose**: This dataset helps us explore customer feedbacks and sentiment and detect market trends.

#### How the Datasets Complement Each Other

The **product information dataset** provides insight into product attributes such as ingredients, pricing and etc., while the **customer reviews dataset** offers feedback on how these products perform from the customer’s perspective. Together, these datasets offer a comprehensive view of the relationship between product attributes and consumer experiences, enabling us to uncover actionable insights for brands in the cosmetic industry. 

It is worth noting that the customer reviews dataset covers reviews on **only 2,000 skincare products**, while the product information dataset includes details on **8,000+ products** across various categories.

## Data Description


In [ ]:
# import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud

### Product Information

Firstly, we load the data from existing `csv` file and get an idea of its shape as well as fields included.

In [ ]:
product_df = pd.read_csv("Sephora/product_info.csv")

In [ ]:
print("product info dataframe shape: ", product_df.shape)
product_df.head()

To get an idea of the general information of the dataset both intuitively and statistically, we use `.info()` and `.describe()` correspondingly to have basic information result, and the outcome is displayed as follows.

In [ ]:
print("Basic Information:")
product_df.info()

In [ ]:
print("\nSummary Statistics (Numerical):")
product_df.describe()

### Customer Review

We load data of customer review into dataframes and concat them together into one complete dataframe altogether. after that, we apply similar analysis methods to have an intuitive overview. The results are displayed as follows.

In [ ]:
# Reference: https://www.kaggle.com/code/themeeemul/sephora-eda-and-sentiment-analysis-using-pytorch
review_df_1 = pd.read_csv("Sephora/reviews_0-250.csv",index_col = 0, dtype={'author_id':'str'})
review_df_2 = pd.read_csv("Sephora/reviews_250-500.csv",index_col = 0, dtype={'author_id':'str'})
review_df_3 = pd.read_csv("Sephora/reviews_500-750.csv",index_col = 0, dtype={'author_id':'str'})
review_df_4 = pd.read_csv("Sephora/reviews_750-1250.csv",index_col = 0, dtype={'author_id':'str'})
review_df_5 = pd.read_csv("Sephora/reviews_1250-end.csv",index_col = 0, dtype={'author_id':'str'})

# Merge review_df_1 till review_df_6
review_df = pd.concat([review_df_1, review_df_2, review_df_3, review_df_4, review_df_5],axis=0)
print("review_df shape: ",review_df.shape)

In [ ]:
review_df.sample(5)

In [ ]:
print("Basic Information:")
review_df.info()

In [ ]:
print("\nSummary Statistics (Numerical):")
review_df.describe()

Note: The missing value part will be introduced in section `Data Cleaning`. 

## Data Manipulation & Visualization

After the first step of importing, merging and describing data, we go a step further into data manipulation, to process on the data and turn it into a more tidy and easy-to-use format. Alongside, we conduct data visualization to provide us a straightforward display of all kinds of data distribution, so as to give us a clearer sense of the data. Aspects include customers' rating, item price, size, etc.

#### Rating

To see the overall reputation condition of products, we visualized the distribution of customers' rating on all products. Shown below we can see that large percentage of customers give a rating between 4.5-5.0/5.0, indicating that they're satisfied with the products.

In [ ]:
# Customer Rating Distribution
plt.figure(figsize=(8, 5))
review_df['rating'].hist(bins=8, color='lightcoral', edgecolor='black')
plt.title('Customer Rating Distribution')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.show()

#### Price

To see the price distribution of all products, we create the visualization below. To avoid outliers, we will only visualize products with price <= 350 USD.

In [ ]:
# Price Distribution
# To avoid outliers, we will only visualize products with price <= 350 USD
plt.figure(figsize=(8, 5))
product_df[product_df['price_usd'] <= 350]['price_usd'].hist(bins=30, color='gold', edgecolor='black')
plt.title('Price Distribution')
plt.xlabel('Price (USD)')
plt.ylabel('Frequency')
plt.show()

#### Size
To see the size of the products, we extract numerical values from certain columns and conduct some datatype conversion to get get the size in both `oz` and `ml`.

In [ ]:
product_df['size']

In [ ]:
product_df['size_oz'] = product_df['size'].str.extract(r'(\d+\.?\d*)\s*oz', expand=False).astype(float)
print(product_df[['size', 'size_oz']].sample(10))

In [ ]:
product_df['size_ml'] = product_df['size'].str.extract(r'(\d+\.?\d*)\s*mL', expand=False)
product_df['size_ml'] = product_df['size_ml'].astype(float)
print(product_df[['size', 'size_ml']].sample(10))

### Price Per Unit

To get an idea of the price distribution of products, we conduct some calculation and visualization to see the distribution. The results are show below.

In [ ]:
product_df['price_per_oz'] = product_df['price_usd'] / product_df['size_oz']
product_df['price_per_ml'] = product_df['price_usd'] / product_df['size_ml']
product_df[['price_usd', 'size_oz', 'price_per_oz', 'size_ml', 'price_per_ml']].sample(10)

In [ ]:
# Price Distribution
# To avoid outliers, we will only visualize products with price <= 350 USD
plt.figure(figsize=(8, 5))
product_df[product_df['price_per_oz'] <= 500]['price_per_oz'].hist(bins=30, color='gold', edgecolor='black')
plt.title('Price Distribution')
plt.xlabel('Price (USD)')
plt.ylabel('Frequency')
plt.show()

### Category

To have an idea of how many items belong to each category, we conduct counting on both primary and secondary categories.

In [ ]:
product_df['primary_category'].value_counts()

In [ ]:
print(product_df['secondary_category'].shape)
product_df['secondary_category'].value_counts()

### Loves Count

In [ ]:
# Price Distribution
# To avoid outliers, we will only visualize products with price <= 350 USD
plt.figure(figsize=(8, 5))
product_df[product_df['loves_count'] < 400000]['loves_count'].hist(bins=30, color='gold', edgecolor='black')
plt.title('loves Count Distribution')
plt.xlabel('Count')
plt.ylabel('Frequency')
plt.show()

### Positive/Negative Feedback Percentage

Also we calculate and visualize on the positive and negative feedback percentages of customers, as another indication of their satisfactory level.

In [ ]:
review_df['pos_feedback_percentage'] = review_df['total_pos_feedback_count']/review_df['total_feedback_count']
review_df['neg_feedback_percentage'] = review_df['total_neg_feedback_count']/review_df['total_feedback_count']
review_df[['total_pos_feedback_count', 'total_feedback_count', 'pos_feedback_percentage', 'total_neg_feedback_count', 'neg_feedback_percentage']].sample(5)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle('Feedback Percentage Distributions')

axes[0].hist(review_df['pos_feedback_percentage'], bins=30, color='green', edgecolor='black')
axes[0].set_title('Positive Feedback Percentage')
axes[0].set_xlabel('Percentage')
axes[0].set_ylabel('Number of Products')

axes[1].hist(review_df['neg_feedback_percentage'], bins=30, color='red', edgecolor='black')
axes[1].set_title('Negative Feedback Percentage')
axes[1].set_xlabel('Percentage')
axes[1].set_ylabel('Number of Products')

plt.tight_layout(rect=[0, 0, 1, 0.95])
plt.show()

### Ingredient
To see the ingredients contained in the products, we processed the strings in the `ingredients` column to extract certain fields, thus making some inferences with the help of visualization tools.

In [ ]:
product_df['ingredients'] = product_df['ingredients'].str.strip("[]").str.replace("'", "").str.split(", ")
all_ingredients = product_df['ingredients'].explode().dropna().tolist()
ingredient_counts = Counter(all_ingredients)
ingredient_df = pd.DataFrame(ingredient_counts.items(), columns=['Ingredient', 'Count']).sort_values(by='Count', ascending=False)

In [ ]:
ingredient_df.head(5)

In [ ]:
top20_ingredients = [ingredient for ingredient, _ in ingredient_counts.most_common(20)]

# Add column in product_df for each of the top 20 ingredients for further analysis
for ingredient in top20_ingredients:
    column_name = f'contains_{ingredient}'
    product_df[column_name] = product_df['ingredients'].apply(lambda x: ingredient in x if isinstance(x, list) else False)

In [ ]:
columns_to_display = ['product_id'] + [f'contains_{ingredient}' for ingredient in top20_ingredients]
product_df[columns_to_display].head()

In [ ]:
plt.figure(figsize=(12, 6))
ingredient_df.head(20).plot(kind='barh', x='Ingredient', y='Count', legend=False, color='skyblue')
plt.title('Top 20 Most Common Ingredients in Products')
plt.xlabel('Count')
plt.ylabel('Ingredient')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(ingredient_counts)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Ingredients')
plt.show()

### Merge Two Datasets

After all of the work and analysis above, we then merge the two datasets together for more convenient future work and analysis. The process is shown below. 

In [ ]:
# Reference: https://www.kaggle.com/code/themeeemul/sephora-eda-and-sentiment-analysis-using-pytorch
# Merge product_df and review_df
cols_to_use = product_df.columns.difference(review_df.columns)
cols_to_use = list(cols_to_use)
cols_to_use.append('product_id')

Sephora_df = pd.merge(review_df, product_df[cols_to_use], how='outer', on=['product_id', 'product_id'])
print("Sephora Shape: ", Sephora_df.shape)

In [ ]:
Sephora_df.sample(5)

In [ ]:
Sephora_df.describe()

### Data Cleaning

Finally, to deal with potential `NaN` values in the data of all kinds, we conduct data cleaning approaches shown as follows. 

In [ ]:
# 1. Check for missing values
print("Missing values per column:")
print(Sephora_df.isnull().sum())

In [ ]:
# 2. Deal with missing values

missing_counts = Sephora_df.isna().sum()
threshold = 1
columns_to_drop = missing_counts[missing_counts > threshold].index
for col in columns_to_drop:
    print(f"{col}: {missing_counts[col]} missing values")

Considering product name the unique identifier without which analysis becomes meaningless, we choose to drop the rows where `product_name` is missing in the first place.

In [ ]:
# Drop rows with no product information (NaN in column 'product_name')
Sephora_df = Sephora_df.dropna(subset=['product_name'])

After that, we fill up the `NaN` values following certain rules correspondingly. The details are displayed below in the code block.

In [ ]:
# 1. Handle feedback-related columns
Sephora_df = Sephora_df.assign(
    helpfulness=Sephora_df['helpfulness'].fillna(0),
    pos_feedback_percentage=Sephora_df['pos_feedback_percentage'].fillna(1),
    neg_feedback_percentage=Sephora_df['neg_feedback_percentage'].fillna(0),
    is_recommended=Sephora_df['is_recommended'].fillna(1)
)

# 2. Handle price-related columns by filling with price_usd
Sephora_df = Sephora_df.assign(
    child_max_price=Sephora_df['child_max_price'].fillna(Sephora_df['price_usd']),
    child_min_price=Sephora_df['child_min_price'].fillna(Sephora_df['price_usd']),
    sale_price_usd=Sephora_df['sale_price_usd'].fillna(Sephora_df['price_usd']),
    value_price_usd=Sephora_df['value_price_usd'].fillna(Sephora_df['price_usd'])
)

# 3. Handle variation-related columns
Sephora_df = Sephora_df.assign(
    variation_desc=Sephora_df['variation_desc'].fillna('No variation'),
    variation_type=Sephora_df['variation_type'].fillna('Unknown'),
    variation_value=Sephora_df['variation_value'].fillna('Unknown')
)

# 4. Handle product and review metadata
Sephora_df = Sephora_df.assign(
    review_text=Sephora_df['review_text'].fillna('No review provided'),
    review_title=Sephora_df['review_title'].fillna('No title'),
    tertiary_category=Sephora_df['tertiary_category'].fillna('Uncategorized')
)

# 5. Handle skin, hair, and eye attributes
Sephora_df = Sephora_df.assign(
    skin_tone=Sephora_df['skin_tone'].fillna('Not specified'),
    eye_color=Sephora_df['eye_color'].fillna('Not specified'),
    skin_type=Sephora_df['skin_type'].fillna('Not specified'),
    hair_color=Sephora_df['hair_color'].fillna('Not specified')
)

# 6. Handle ingredient and highlight columns
Sephora_df = Sephora_df.assign(
    ingredients=Sephora_df['ingredients'].fillna('Ingredients not listed'),
    highlights=Sephora_df['highlights'].fillna('No highlights available')
)

# 7. Handle size-related columns
Sephora_df = Sephora_df.assign(
    size=Sephora_df['size'].fillna('Unknown size'),
    size_ml=Sephora_df['size_ml'].fillna(Sephora_df['size_ml'].median()),
    size_oz=Sephora_df['size_oz'].fillna(Sephora_df['size_oz'].median())
)

# 8. Handle price per unit columns
Sephora_df = Sephora_df.assign(
    price_per_ml=Sephora_df['price_per_ml'].fillna(Sephora_df['price_per_ml'].median()),
    price_per_oz=Sephora_df['price_per_oz'].fillna(Sephora_df['price_per_oz'].median())
)

# 9. Print the result to confirm the missing values have been handled
missing_counts_after = Sephora_df.isna().sum()
print("Missing values after processing:")
print(missing_counts_after[missing_counts_after > 0])

In [ ]:
numerical_cols = Sephora_df.select_dtypes(include=['float64', 'int64']).columns
categorical_cols = Sephora_df.select_dtypes(include=['object']).columns

In [ ]:
# 3. Strip whitespace from string columns (if any)
Sephora_df[categorical_cols] = Sephora_df[categorical_cols].apply(lambda x: x.str.strip())

## Part 2

In Project Part 2, we will focus on applying statistical and graphical analyses to extract insights and address the research questions we posed. 

For review, this is our research questions:
1. What factors drive higher product ratings? Why do certain products underperform?  
2. How does customer sentiment change over time?
3. Are certain ingredients linked to positive or negative customer experiences?

### Research Problem 1: **What factors drive higher product ratings? Why do certain products underperform?**
For research problem 1, our primary interest lies in understanding how **product-related factors** influence **average ratings of the product**. Therefore we will mainly focus on **product_df**. The combined dataset will be used in research problem 2 & research problem 3.

#### Supplement Project Part 1: Basic Analysis

This part we will supplement some basic analysis of 

In [ ]:
product_df.columns

To analyze the relationship between product average ratings and related numerical values (such as Price_usd, Price_per_oz, Size_oz, Loves_count, and Reviews), we can calculate the correlation between rating and these numerical features in the product_df DataFrame. This will help us identify which factors are most strongly associated with ratings.

In [ ]:
correlation_features = ['rating', 'price_usd', 'price_per_oz', 'size_oz', 'loves_count', 'reviews']

correlation_data = product_df[correlation_features]

In [ ]:
corr_matrix = correlation_data.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Relationship between rating and top 10 ingredients
correlation_features = ['rating', 'contains_Glycerin',
       'contains_Phenoxyethanol', 'contains_Caprylyl Glycol',
       'contains_Tocopherol', 'contains_Limonene',
       'contains_Ethylhexylglycerin', 'contains_Dimethicone',
       'contains_Linalool', 'contains_Silica', 'contains_Butylene Glycol']
correlation_data = product_df[correlation_features]

In [ ]:
corr_matrix = correlation_data.corr()

plt.figure(figsize=(10, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

From the results above, we can observe that there is **no significant relationship** between loves_count, reviews, ingredients, and rating. 

However, **we will analyze the numerical variables in more detail** in the next several sections, and also **analyze the categorical variables using the Comparative & Multivariate Analysis**.

#### Descriptive Statistics

- **Objective**: Use descriptive statistics to explore the central tendency and dispersion of rating data to identify primary drivers of product ratings.
- **Method**: Calculate the mean, median, mode, range, variance, and standard deviation of product ratings and other factors.
- **Expected Outcome**: By analyzing the mean and median, we can assess rating distribution. Categories with higher standard deviations might indicate a broader range of factors affecting performance.

In [ ]:
# Use ChatGPT to generate a summary of the dataset

# Descriptive statistics for the 'rating' column and other relevant columns in product_df
rating_data = product_df['rating']  # Replace 'rating' with actual column name if different

# Calculating descriptive statistics
mean_rating = rating_data.mean()
median_rating = rating_data.median()
mode_rating = rating_data.mode().iloc[0]  # Mode may return multiple values; take the first mode
range_rating = rating_data.max() - rating_data.min()
variance_rating = rating_data.var()
std_dev_rating = rating_data.std()

# Print results
print("Descriptive Statistics for Ratings:")
print(f"Mean: {mean_rating}")
print(f"Median: {median_rating}")
print(f"Mode: {mode_rating}")
print(f"Range: {range_rating}")
print(f"Variance: {variance_rating}")
print(f"Standard Deviation: {std_dev_rating}")

**Conclusions:**

1. **Mean and Median**: 
   - The mean (4.19) and median (4.29) reflect generally positive product ratings, with a skew toward higher values, indicating strong overall customer satisfaction.

2. **Mode**: 
   - A mode of 5.0 suggests that a significant number of products received top ratings.

3. **Range**:
   - Average product ratings span from 1.0 to 5.0, indicating differences in customer satisfaction.

4. **Variance and Standard Deviation**: 
   - The moderate variance (0.27) and standard deviation (0.52) indicate some variation in ratings, showing that while most ratings are clustered, there is notable diversity.

In [ ]:
# If you want to calculate these metrics for additional columns (e.g., 'price', 'review_count', etc.), you can loop through them:
relevant_columns = ['price_usd', 'price_per_oz', 'size_oz', 'loves_count', 'reviews']  # Replace with actual column names

# Loop through relevant columns and print statistics
for column in relevant_columns:
    print(f"\nDescriptive Statistics for {column.capitalize()}:")
    mean = product_df[column].mean()
    median = product_df[column].median()
    mode = product_df[column].mode().iloc[0]  # Take the first mode if multiple
    value_range = product_df[column].max() - product_df[column].min()
    variance = product_df[column].var()
    std_dev = product_df[column].std()
    
    print(f"Mean: {mean}")
    print(f"Median: {median}")
    print(f"Mode: {mode}")
    print(f"Range: {value_range}")
    print(f"Variance: {variance}")
    print(f"Standard Deviation: {std_dev}")

**Conclusions and Why they are considered as related factors:**

1. **Price_usd**: 
   - Positive skew with a mean of $51.66 and a high variance suggests a range of product pricing. It will potentially impact ratings if high-priced items don't meet expectations.

2. **Price_per_oz**: 
   - The high mean of $160.71 with a large standard deviation implies a few very expensive products that may set variable customer expectations and influence ratings.

3. **Size_oz**: 
   - Mean of 3.63 oz and lower variance suggest that most products are small, which could indirectly impact ratings if larger sizes are perceived as better value.

4. **Loves_count**: 
   - High variance and mean of 29,179 suggest popularity varies widely. Higher "Loves" counts may correlate with higher ratings, reflecting customer satisfaction.

5. **Reviews**: 
   - Positive skew with a mean of 448.55 and high variance indicates some products have more customer engagement, which could stabilize average ratings.

#### Inferential Statistics

- **Objective**: Validate which variables (such as price, product type, or ingredients) significantly influence product ratings.
- **Method**:
    - **Regression Analysis**: Evaluate relationships between product features (e.g., price, number of ingredients) and ratings.
    - **ANOVA (Analysis of Variance)**: Test for significant differences in ratings across different product categories.
- **Expected Outcome**: Determine whether certain product features have a significant effect on ratings, explaining why some products underperform.

##### Regression Analysis

This is for testing correlation with continuous variables like `Price_usd`

**Steps**:
1. **Define Hypotheses**:
   - **Null Hypothesis (H₀)**: There is no significant relationship between product ratings and price (`Price_usd`).
   - **Alternative Hypothesis (H₁)**: There is a significant relationship between product ratings and price.

2. **Conduct Regression**:
   - Perform a simple linear regression with `rating` as the dependent variable and `Price_usd` as the independent variable.
   - If you have multiple factors (like `Price_usd`, `Price_per_oz`, etc.), use multiple linear regression.

3. **Choose Significance Level**:
   - Typically, we use a significance level of α = 0.05.

4. **Interpret Results**:
   - If the p-value for `Price_usd` (or other factors) is below 0.05, reject the null hypothesis, suggesting a significant relationship between rating and price.
   - Check the coefficient to see if the relationship is positive or negative.

#### Graphical Analysis

- **Objective**: Visually display rating distributions and correlations to identify potential drivers.
- **Method**: Use bar charts and box plots to observe rating distributions across product categories, and scatter plots to examine ratings against factors like price and ingredients.
- **Expected Outcome**: Identify trends in rating distribution and potential outliers to determine which features are associated with higher or lower ratings.

##### Price v.s. Rating

In [ ]:
# Hexbin Plot (Rating vs Price)
# Filter products with price <= 350 USD
filtered_df = product_df[product_df['price_usd'] <= 350]

plt.figure(figsize=(8, 6))
plt.hexbin(filtered_df['price_usd'], filtered_df['rating'], gridsize=30, 
           cmap='viridis', mincnt=1)

plt.colorbar(label='Number of Products')
plt.xlabel('Price (USD)')
plt.ylabel('Rating')
plt.title('Hexbin Plot of Rating vs Price (<= 350 USD)')
plt.show()

In [ ]:
# Hexbin Plot (Rating vs Unit Price)
# Filter products with unit price <= 600 USD
filtered_df = product_df[product_df['price_per_oz'] <= 600]

plt.figure(figsize=(8, 6))
plt.hexbin(filtered_df['price_per_oz'], filtered_df['rating'], gridsize=30, 
           cmap='viridis', mincnt=1)

plt.colorbar(label='Number of Products')
plt.xlabel('Price (USD)')
plt.ylabel('Rating')
plt.title('Hexbin Plot of Rating vs Price (<= 350 USD)')
plt.show()

In [ ]:
bins = [0, 10, 20, 50, 100, 200, 500, product_df['price_usd'].max()]
labels = ['0-10', '10-20', '20-50', '50-100', '100-200', '200-500', f'500+']

product_df['price_bin'] = pd.cut(product_df['price_usd'], bins=bins, labels=labels, include_lowest=True)

avg_rating_by_bin = product_df.groupby('price_bin')['rating'].mean().reset_index()

plt.figure(figsize=(6, 4))
plt.bar(avg_rating_by_bin['price_bin'], avg_rating_by_bin['rating'])
plt.title('Average Rating by Price Bin', fontsize=16)
plt.xlabel('Price Range (USD)', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)

plt.show()

In [ ]:
bins = [0, 50, 100, 200, 500, product_df['price_per_oz'].max()]
labels = ['0-50', '50-100', '100-200', '200-500', f'500+']

product_df['price_bin'] = pd.cut(product_df['price_per_oz'], bins=bins, labels=labels, include_lowest=True)

avg_rating_by_bin = product_df.groupby('price_bin')['rating'].mean().reset_index()

plt.figure(figsize=(6, 4))
plt.bar(avg_rating_by_bin['price_bin'], avg_rating_by_bin['rating'])
plt.title('Average Rating by Price Bin', fontsize=16)
plt.xlabel('Price Range (USD)', fontsize=12)
plt.ylabel('Average Rating', fontsize=12)

plt.show()

**Analysis:**

From the figure above, we can observe **slight differences between total price and average rating**. However, **no noticeable differences** are found between unit price and average rating.  

Additionally, if we aim to **achieve higher ratings**, setting the price within the **20-200** range may be effective.

#### Comparative Analysis

- **Objective**: Compare ratings across different product categories or other subsets to explore variations.
- **Method**: Compare ratings by category to identify the performance of specific groups.
- **Expected Outcome**: Identify categories with notably higher or lower ratings, explaining which factors might drive ratings up or down.

##### Category v.s. Rating

In [ ]:
# Primary Category vs Average Rating
product_df['rating'] = pd.to_numeric(product_df['rating'], errors='coerce')

avg_rating_by_category = product_df.groupby('primary_category')['rating'].mean().\
    reset_index().sort_values(by='rating', ascending=False)
avg_rating_by_category.columns = ['primary_category', 'average_rating']
print(avg_rating_by_category)

In [ ]:
# Secondary Category vs Average Rating
avg_rating_by_category = product_df.groupby('secondary_category')['rating'].mean().\
    reset_index().sort_values(by='rating', ascending=False)
avg_rating_by_category.columns = ['secondary_category', 'average_rating']
print(avg_rating_by_category)

In [ ]:
# Secondary Category vs Average Rating
avg_rating_by_category = product_df.groupby('tertiary_category')['rating'].mean().\
    reset_index().sort_values(by='rating', ascending=False)
avg_rating_by_category.columns = ['tertiary_category', 'average_rating']
print(avg_rating_by_category)

**Analysis:**

From the results, we can observe the following trends:

**Primary Category:**  
- **Top performers:** "Gifts" (4.56) and "Men" (4.50) have the highest average ratings, indicating that these product categories are highly appreciated by customers. This may suggest that these products meet customer expectations well, potentially due to their quality, usability, or alignment with specific customer needs.  
- **Lower performers:** "Mini Size" (4.00) and "Makeup" (4.14) receive comparatively lower ratings, suggesting room for improvement. It’s possible that mini-sized products offer less value or that makeup items may not fully meet customer expectations in terms of variety, durability, or performance.  

**Secondary Category:**  
- **Highest rated:** "Other Needs" (4.68) and "Shaving" (4.64) receive the highest scores, indicating strong customer satisfaction. Shaving products might perform well due to their practical utility, while "Other Needs" likely covers niche items that satisfy specific demands.  
- **Lower ratings:** Categories like "Nail" (3.88) and "Fragrance" (3.87) are rated lower, possibly due to limited product effectiveness, insufficient variety, or unmet customer expectations. The subjective nature of fragrances and varying preferences in nail care products could contribute to these lower ratings.  

**Tertiary Category:**  
- **Top categories:** "Lip Brushes" (4.71) and "Shaving" (4.71) are among the highest-rated, suggesting these niche products perform well. This could imply that customers appreciate specialized tools that enhance their beauty routines or provide value through functionality.  
- **Lowest performers:** "Manicure & Pedicure Tools" (2.29) and "BB & CC Cream" (3.67) have the lowest average ratings, indicating dissatisfaction or unmet expectations. This could result from poor product quality, lack of innovation, or insufficient effectiveness compared to competitors.  

These results highlight areas where certain products excel and others fall short, suggesting that focusing on **customer feedback and product refinement** in the lower-performing categories could enhance overall satisfaction.

##### Brand vs Average Rating

In [ ]:
# brand_name vs Average Rating
avg_rating_by_category = product_df.groupby('brand_name')['rating'].mean().\
    reset_index().sort_values(by='rating', ascending=False)
avg_rating_by_category.columns = ['brand_name', 'average_rating']
print(avg_rating_by_category)

**Analysis:**

**Top Brands:**
- **Erno Laszlo** (5.00), **Aquis** (4.90), **MACRENE actives** (4.89), and **MARA** (4.82) show high customer satisfaction, likely due to consistent quality and meeting customer needs.

**Lower-Rated Brands:**
- **Christophe Robin** (3.15) and **Good Dye Young** (3.06) have lower ratings, suggesting issues with product quality or unmet expectations.

**Lowest-Rated Brand:**
- **Overose** (2.85) indicates significant dissatisfaction, likely due to performance issues or poor customer experience.

High-performing brands should maintain their quality, while underperforming brands need to address product and customer experience issues.

##### Rating v.s. Online or not

In [ ]:
# Online vs Average Rating
avg_rating_by_category = product_df.groupby('online_only')['rating'].mean().\
    reset_index().sort_values(by='rating', ascending=False)
avg_rating_by_category.columns = ['online_only', 'average_rating']
print(avg_rating_by_category)

**Non-online products** (4.20) have slightly higher ratings than **online-only products** (4.17).

#### Multivariate Analysis

- **Objective**: Study the influence of multiple factors on ratings simultaneously.
- **Method**: Use pivot tables and multivariate scatter plots to explore the relationship between ingredient counts, prices, and ratings.
- **Expected Outcome**: Understand how various factors interact to drive ratings higher or lower.

##### Rating v.s. Online or not & Category

In [ ]:
# Calculate average rating by both primary category and online_only status
avg_rating_online_category = product_df.groupby(['primary_category', 'online_only'])['rating'].mean().reset_index()

# Pivot the data for better comparison
pivot_table = avg_rating_online_category.pivot(index='primary_category', columns='online_only', values='rating')

pivot_table.columns = ['Non-Online', 'Online']

# Calculate the rating difference between online and non-online categories
pivot_table['Rating_Difference'] = pivot_table['Online'] - pivot_table['Non-Online']

pivot_table = pivot_table.sort_values(by='Rating_Difference', ascending=False)
print(pivot_table)

**Analysis:**

**Primary Category Comparison:**  
1. **Higher Online Ratings:**  
   - **Mini Size:** 4.06 vs. 3.99 (+0.06) – More convenient for online purchases.  
   - **Skincare:** 4.27 vs. 4.22 (+0.05) – Benefits from detailed product information online.

2. **Minimal Difference:**  
   - **Tools & Brushes:** 4.27 vs. 4.27 (-0.0005) – Similar satisfaction across both channels.

3. **Higher Non-Online Ratings:**  
   - **Makeup:** 4.16 vs. 3.99 (-0.17) – In-person trials likely preferred.  
   - **Fragrance:** 4.26 vs. 4.14 (-0.11) – Harder to evaluate online.  
   - **Men:** 4.54 vs. 4.38 (-0.16) – Benefits from in-store accessibility and recommendations.

**Conclusion:**
- **Products like Mini Size and Skincare** benefit from online availability, likely due to convenience and access to reviews.
- **Makeup and Fragrances** perform better offline, suggesting that in-store experiences play a key role in customer satisfaction for these categories.
- **Distribution Strategy Implication:** Focusing on enhancing **online shopping experiences for Makeup and Fragrances** (e.g., offering virtual try-ons) could help improve ratings.

#### Synthesis

**Conclusion:**


### Research Question 2: **How does customer sentiment change over time?**

1. **Descriptive Statistics**:
    - **Objective**: Explore trends in time series data through descriptive statistics to gain an initial understanding of customer sentiment fluctuations.
    - **Method**: Calculate the mean and standard deviation of sentiment ratings for each time period to determine the trend and range of sentiment changes.
    - **Expected Outcome**: Identify central tendencies and volatility in sentiment, providing an initial understanding of sentiment trends over time.

2. **Inferential Statistics**:
    - **Objective**: Test the significance of time-related changes in customer sentiment.
    - **Method**: Use **time series analysis** or **linear regression** to assess the influence of time on sentiment ratings.
    - **Expected Outcome**: Determine if sentiment changes significantly over time, offering potential explanations for sentiment shifts.

3. **Graphical Analysis**:
    - **Objective**: Use visualizations to identify sentiment change patterns within the time series.
    - **Method**: Use line charts and bar charts to display sentiment score changes over different periods, making trends easier to observe.
    - **Expected Outcome**: Visually capture how sentiment changes over time, identifying possible fluctuations or significant trends.

4. **Comparative Analysis**:
    - **Objective**: Compare sentiment ratings across different time periods to observe if there are notable differences.
    - **Method**: Group data by time segments and compare changes in sentiment ratings to identify any seasonal or periodic trends.
    - **Expected Outcome**: Discover significant sentiment changes during specific periods, offering insights into trends in customer sentiment.

5. **Multivariate Analysis**:
    - **Objective**: Investigate the combined effect of time and other variables (e.g., ingredients, ratings) on sentiment.
    - **Method**: Use cross-tabulations and multivariate charts to analyze sentiment score variations across different ingredients over time.
    - **Expected Outcome**: Identify complex relationships between sentiment scores, time, and other variables, determining if sentiment changes are influenced by additional factors.

---

### Research Question 3: **Are certain ingredients linked to positive or negative customer experiences?**

1. **Descriptive Statistics**:
    - **Objective**: Use descriptive statistics to explore associations between ingredients and customer experience scores.
    - **Method**: Calculate the mean, variance, and standard deviation of sentiment ratings for products containing specific ingredients to gauge their effects.
    - **Expected Outcome**: Observe if specific ingredients are associated with higher or lower sentiment scores, offering a preliminary indication of their impact on customer experience.

2. **Inferential Statistics**:
    - **Objective**: Test the significant relationship between specific ingredients and customer experience.
    - **Method**: Use **chi-squared tests** to assess the relationship between ingredients and sentiment scores, or apply **t-tests** and **ANOVA** to compare ratings between products with and without certain ingredients.
    - **Expected Outcome**: Confirm whether certain ingredients are significantly associated with differences in customer experience, supporting or refuting their influence on sentiment.

3. **Graphical Analysis**:
    - **Objective**: Visually display the relationship between ingredients and customer experience.
    - **Method**: Use bar charts and box plots to display sentiment score distributions for products with specific ingredients, helping to identify patterns.
    - **Expected Outcome**: View the distribution of ratings for products with various ingredients, revealing ingredients potentially linked to positive or negative experiences.

4. **Comparative Analysis**:
    - **Objective**: Compare sentiment scores among different ingredient combinations to explore ingredient-related differences.
    - **Method**: Compare sentiment ratings for products with different ingredient combinations to identify significant differences.
    - **Expected Outcome**: Identify ingredient combinations associated with notably positive or negative experiences, providing insights into ingredient impact on customer sentiment.

5. **Multivariate Analysis**:
    - **Objective**: Understand the combined relationships between ingredients, ratings, and sentiment.
    - **Method**: Use multivariate scatter plots and cross-tabulations to analyze relationships between ingredients, ratings, and sentiment scores.
    - **Expected Outcome**: Determine how multiple ingredients collectively influence customer experience, identifying potential interaction effects. 


### Synthesis

### Reflection